# Q&A against Tabular Data from a CSV file

To really have a Smart Search Engine or Virtual assistant that can answer any question about your corporate documents, this "engine" must understand tabular data, aka, sources with tables, rows and columns with numbers. 
This is a different problem that simply looking for the top most similar results.  The concept of indexing, bringing top results, embedding, doing a cosine semantic search and summarize an answer, doesn't really apply to this problem.
We are dealing now with sources with Tables in which each row and column are related to each other, and in order to answer a question, all of the data is needed, not just top results.

In this notebook, the goal is to show how to deal with this kind of use cases. To continue with our Covid-19 theme, we will be using an open dataset called ["Covid Tracking Project"](https://learn.microsoft.com/en-us/azure/open-datasets/dataset-covid-tracking?tabs=azure-storage). The COVID Tracking Project dataset is a  CSV file that provides the latest numbers on tests, confirmed cases, hospitalizations, and patient outcomes from every US state and territory (they stopped tracking on March 7 2021).

Imagine that many documents on a data lake are tabular data, or that your use case is to ask questions in natural language to a LLM model and this model needs to get the context from a CSV file or even a SQL Database in order to answer the question. A GPT Smart Search Engine, must understand how to deal with this sources, understand the data and answer acoordingly.

In [99]:
import os
import pandas as pd
from langchain.llms import AzureOpenAI
from langchain.chat_models import AzureChatOpenAI
from langchain.agents import create_pandas_dataframe_agent
from langchain.agents import create_csv_agent

# Don't mess with this unless you really know what you are doing
AZURE_OPENAI_API_VERSION = "2023-03-15-preview"

# Change these below with your own services credentials
AZURE_OPENAI_ENDPOINT = "https://open-ai-pinternal.openai.azure.com/"
AZURE_OPENAI_KEY = "b2dc4d69945240a993288400d3b18faf"

# AZURE_OPENAI_ENDPOINT = "Enter your Azure OpenAI Endpoint ..."
# AZURE_OPENAI_KEY = "Enter your Azure OpenAI Key ..."

In [100]:
# Set the ENV variables that Langchain needs to connect to Azure OpenAI
os.environ["OPENAI_API_BASE"] = os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ENDPOINT
os.environ["OPENAI_API_KEY"] = os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_KEY
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"] = AZURE_OPENAI_API_VERSION

## Download the dataset and load it into Pandas Dataframe

In [101]:
file_url = "https://pandemicdatalake.blob.core.windows.net/public/curated/covid-19/covid_tracking/latest/covid_tracking.csv"
df = pd.read_csv(file_url).fillna(value = 0)
print("Rows and Columns:",df.shape)
df.head()

Rows and Columns: (22261, 31)


,date,state,positive,negative,pending,hospitalized_currently,hospitalized_cumulative,in_icu_currently,in_icu_cumulative,on_ventilator_currently,on_ventilator_cumulative,recovered,data_quality_grade,last_update_et,hash,date_checked,death,hospitalized,total,total_test_results,pos_neg,fips,death_increase,hospitalized_increase,negative_increase,positive_increase,total_test_results_increase,fips_code,iso_subdivision,load_time,iso_country
0,2021-03-07,AK,56886.0,0.0,0.0,33.0,1293.0,0.0,0.0,2.0,0.0,0.0,0.0,2021-03-05 03:59:00,dc4bccd4bb885349d7e94d6fed058e285d4be164,2021-03-05T03:59:00Z,305.0,1293.0,56886,1731628.0,56886,2,0,0,0,0.0,0,2,US-AK,2023-04-13 00:05:32,US
1,2021-03-07,AL,499819.0,1931711.0,0.0,494.0,45976.0,0.0,2676.0,0.0,1515.0,295690.0,0.0,2021-03-07 11:00:00,997207b430824ea40b8eb8506c19a93e07bc972e,2021-03-07T11:00:00Z,10148.0,45976.0,2431530,2323788.0,2431530,1,-1,0,2087,408.0,2347,1,US-AL,2023-04-13 00:05:32,US
2,2021-03-07,AR,324818.0,2480716.0,0.0,335.0,14926.0,141.0,0.0,65.0,1533.0,315517.0,0.0,2021-03-07 00:00:00,50921aeefba3e30d31623aa495b47fb2ecc72fae,2021-03-07T00:00:00Z,5319.0,14926.0,2805534,2736442.0,2805534,5,22,11,3267,165.0,3380,5,US-AR,2023-04-13 00:05:32,US
3,2021-03-07,AS,0.0,2140.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-12-01 00:00:00,f77912d0b80d579fbb6202fa1a90554fc4dc1443,2020-12-01T00:00:00Z,0.0,0.0,2140,2140.0,2140,60,0,0,0,0.0,0,60,US-AS,2023-04-13 00:05:32,US
4,2021-03-07,AZ,826454.0,3073010.0,0.0,963.0,57907.0,273.0,0.0,143.0,0.0,0.0,0.0,2021-03-07 00:00:00,0437a7a96f4471666f775e63e86923eb5cbd8cdf,2021-03-07T00:00:00Z,16328.0,57907.0,3899464,7908105.0,3899464,4,5,44,13678,1335.0,45110,4,US-AZ,2023-04-13 00:05:32,US


In [102]:
df.columns

Index(['date', 'state', 'positive', 'negative', 'pending',
       'hospitalized_currently', 'hospitalized_cumulative', 'in_icu_currently',
       'in_icu_cumulative', 'on_ventilator_currently',
       'on_ventilator_cumulative', 'recovered', 'data_quality_grade',
       'last_update_et', 'hash', 'date_checked', 'death', 'hospitalized',
       'total', 'total_test_results', 'pos_neg', 'fips', 'death_increase',
       'hospitalized_increase', 'negative_increase', 'positive_increase',
       'total_test_results_increase', 'fips_code', 'iso_subdivision',
       'load_time', 'iso_country'],
      dtype='object')

## Load our LLM and create our MRKL Agent

The implementation of Agents is inspired by two papers: the [MRKL Systems](https://arxiv.org/abs/2205.00445) paper (pronounced ‘miracle’ 😉) and the [ReAct](https://arxiv.org/abs/2210.03629) paper.

Agents are a way to leverage the ability of LLMs to understand and act on prompts. In essence, an Agent is an LLM that has been given a very clever initial prompt. The prompt tells the LLM to break down the process of answering a complex query into a sequence of steps that are resolved one at a time.

Agents become really cool when we combine them with ‘experts’, introduced in the MRKL paper. Simple example: an Agent might not have the inherent capability to reliably perform mathematical calculations by itself. However, we can introduce an expert - in this case a calculator, an expert at mathematical calculations. Now, when we need to perform a calculation, the Agent can call in the expert rather than trying to predict the result itself. This is actually the concept behind [ChatGPT Pluggins](https://openai.com/blog/chatgpt-plugins).

In our case, in order to solve the problem "How do I ask questions to a tabular CSV file", we need this REACT/MRKL approach, in which we need to instruct the LLM that it needs to use an 'expert/tool' in order to read/load/understand/interact with a CSV tabular file.

OpenAI opened the world to a whole new concept. Libraries are being created fast and furious. We will be using [LangChain](https://docs.langchain.com/docs/) as our library to solve this problem, however there are others that we recommend: [HayStack](https://haystack.deepset.ai/) and [Semantic Kernel](https://learn.microsoft.com/en-us/semantic-kernel/whatissk).

In [103]:
# First we load our LLM: GPT-4 (you are welcome to try GPT-3.5-Turbo. 
# You will see that it does not have the cognitive capabilities to become a smart agent)

llm = AzureChatOpenAI(deployment_name="gpt-4", temperature=0.5, max_tokens=500)

In [104]:
# Now we need our agent and our expert/tool.  
# LangChain has created an out-of-the-box agents that we can use to solve our Q&A to CSV tabular data file problem. 
agent = create_csv_agent(llm, file_url, verbose=True)

In [109]:
# Natural Language question (query) + a Suffix to format the answer the way we like
preffix = 'First set the pandas display options to show all the columns, then get the column names, then answer the question: '
query_str = 'How may patients were hospitalized during July 2020 in Texas and to nationwide?'
suffix = '. ALWAYS before giving the Final Answer, try another method. Then reflect on the answers of the two methods you did and ask yourself if it answers correctly the original question. If you are not sure, try another method. \n If the runs does not give the same result, reflect and try again two more times until you have two runs that have the same result. If you still cannot arrive to a consistent result, say that you are not sure of the answer. But, if you are sure of the correct answer, create a beautiful and thorough response. ALWAYS, as part of your final answer, explain how you got to the answer on a section that starts with: "\n\nExplanation:\n."'

## Enjoy the response and the power of GPT-4 + REACT/MKRL approach

In [110]:
# We are doing a for loop to retry N times. This is because: 
# 1) GPT-4 is still in preview and the API is being very throttled and 
# 2) Because the LLM not always gives the answer on the exact format the agent needs and hence cannot be parsed

for i in range(5):
    try:
        response = agent.run(query_str + suffix) 
        break
    except:
        response = "Error too many failed retries"
        continue  
        
print(response)



> Entering new AgentExecutor chain...
Thought: First, I need to filter the dataframe to only contain records from Texas and records from July 2020. Then, I'll compute the sum of the 'hospitalized_cumulative' column.
Action: python_repl_ast
Action Input: df.info()<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22261 entries, 0 to 22260
Data columns (total 31 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         22261 non-null  datetime64[ns]
 1   state                        22261 non-null  object        
 2   positive                     22066 non-null  float64       
 3   negative                     14225 non-null  float64       
 4   pending                      2479 non-null   float64       
 5   hospitalized_currently       18584 non-null  float64       
 6   hospitalized_cumulative      13084 non-null  float64       
 7   in_icu_currently             12655 n

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Requests to the Creates a completion for the chat message Operation under Azure OpenAI API version 2023-03-15-preview have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 3 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit..


Now that the date column is in a datetime format, I can try filtering the dataframe again to only have records from Texas and records from July 2020.
Action: python_repl_ast
Action Input: tx_july_2020 = df[(df['state'] == 'TX') & (df['date'].dt.month == 7) & (df['date'].dt.year == 2020)]
tx_july_2020
Observation:             date state  positive  negative  pending  hospitalized_currently  \
13190 2020-07-31    TX  420946.0       NaN      NaN                  9336.0   
13250 2020-07-30    TX  412107.0       NaN      NaN                  9296.0   
13310 2020-07-29    TX  403307.0       NaN      NaN                  9595.0   
13370 2020-07-28    TX  394265.0       NaN      NaN                  9593.0   
13430 2020-07-27    TX  385923.0       NaN      NaN                 10893.0   
13490 2020-07-26    TX  381656.0       NaN      NaN                 10893.0   
13550 2020-07-25    TX  375846.0       NaN      NaN                 10893.0   
13610 2020-07-24    TX  369826.0       NaN      NaN  

# Summary

So, we just solved our problem on how to ask questions in natural language to our Tabular data hosted on a CSV File.
With this approach you can see then that it is NOT necessary to make a dump of a database data into a CSV file and index that on a Search Engine, you don't even need to use the above approach and deal with a CSV data dump file. With the Agents framework, the best engineering decision is to interact directly with the data source API without the need to replicate the data in order to ask questions to it. Remember, GPT-4 can do SQL very well. 

Just think about this: if GPT-4 can do the above, imagine what GPT-5/6/7/8 will be able to do.

This is implemented in the App of this repo as well, where you will be able to upload any tabular CSV you like and ask questions to it

# Reference

- https://haystack.deepset.ai/blog/introducing-haystack-agents
- https://python.langchain.com/en/latest/modules/agents/agents.html
- https://tsmatz.wordpress.com/2023/03/07/react-with-openai-gpt-and-langchain/
- https://medium.com/@meghanheintz/intro-to-langchain-llm-templates-and-agents-8793f30f1837

# NEXT
We can see that GPT-4 is powerful and can translate a natural language question into the right steps in python in order to query a CSV data loaded into a pandas dataframe. 
That's pretty amazing. However the question remains: **Do I need then to dump all the data from my original sources (Databases, ERP Systems, CRM Systems) in order to be searchable by a Smart Search Engine?**

The next notebook 5 answers this question by implementing a Question->SQL process and get the information from data in a SQL Database, eliminating the need to dump it out.